In [1]:
%matplotlib inline
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict
import operator
np.set_printoptions(suppress=True)
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv('C:/Users/maria/OneDrive/Desktop/Smart_Meter_Data/New1.txt', sep=" ")
data.drop('date', axis=1, inplace=True)
data.drop('month', axis=1, inplace=True)

In [5]:
data['date'] = pd.to_datetime(data['day'], unit='D',
                              origin=pd.Timestamp('2008-12-31')).dt.strftime('%d-%m-%y')
data['month'] = pd.to_datetime(data['date'],format='%d-%m-%y').dt.month

In [8]:
data[data.month==10]

,id,demand,day,period,dow,tow,week,date,month
3792,1002,0.588,274,1,4,145,11,01-10-09,10
3793,1002,0.655,274,2,4,146,11,01-10-09,10
3794,1002,0.319,274,3,4,147,11,01-10-09,10
3795,1002,0.061,274,4,4,148,11,01-10-09,10
3796,1002,0.147,274,5,4,149,11,01-10-09,10
...,...,...,...,...,...,...,...,...,...
16739701,1997,0.567,669,44,7,332,67,31-10-10,10
16739702,1997,0.506,669,45,7,333,67,31-10-10,10
16739703,1997,0.536,669,46,7,334,67,31-10-10,10
16739704,1997,0.551,669,47,7,335,67,31-10-10,10


In [26]:
data.reset_index(drop=True, inplace=True)

In [27]:
data

,id,demand,day,period,dow,tow,week,date,month
0,1002,0.522,201,1,1,1,1,20-07-09,7
1,1002,0.349,201,2,1,2,1,20-07-09,7
2,1002,0.102,201,3,1,3,1,20-07-09,7
3,1002,0.101,201,4,1,4,1,20-07-09,7
4,1002,0.035,201,5,1,5,1,20-07-09,7
...,...,...,...,...,...,...,...,...,...
16181467,1997,0.979,725,44,7,332,75,26-12-10,12
16181468,1997,1.023,725,45,7,333,75,26-12-10,12
16181469,1997,0.672,725,46,7,334,75,26-12-10,12
16181470,1997,0.760,725,47,7,335,75,26-12-10,12


In [28]:
file_name = "New2.txt"
file_loc = r"C:/Users/maria/OneDrive/Desktop/Smart_Meter_Data/" + file_name
data.to_csv(file_loc, index=None, sep=' ', mode='a')

In [13]:
data.drop(data[data.day==452].index, inplace=True)
data.drop(data[data.week==0].index, inplace=True) # first week started on day 201
data.drop(data[data.week==36].index, inplace=True)# for now delete the week which contains day 452
data.drop(data[data.week==76].index, inplace=True) # last week is week 75

In [14]:
all_id = data.id.unique()
print(len(all_id))
id_lst = []
for x in all_id:
    if len(data[data['id'] == x]) == 24864:
        id_lst.append(x)
print(len(id_lst))

651
556


In [15]:
residential_series = data.id.isin(id_lst)
df = data[residential_series]

df_z_scaled = pd.DataFrame()
column = 'demand'
for x in id_lst:
    df_temp = df[(df.id == x)]
    df_temp[column] = (df_temp[column] - df_temp[column].mean()) / df_temp[column].std()
    df_z_scaled = df_z_scaled.append(df_temp, ignore_index = True)

In [19]:
df.reset_index(drop=True, inplace=True)

In [20]:
df

,id,demand,day,period,dow,tow,week,date,month
0,1002,0.522,201,1,1,1,1,20-07-09,7
1,1002,0.349,201,2,1,2,1,20-07-09,7
2,1002,0.102,201,3,1,3,1,20-07-09,7
3,1002,0.101,201,4,1,4,1,20-07-09,7
4,1002,0.035,201,5,1,5,1,20-07-09,7
...,...,...,...,...,...,...,...,...,...
13824379,1997,0.979,725,44,7,332,75,26-12-10,12
13824380,1997,1.023,725,45,7,333,75,26-12-10,12
13824381,1997,0.672,725,46,7,334,75,26-12-10,12
13824382,1997,0.760,725,47,7,335,75,26-12-10,12


In [21]:
df_z_scaled.reset_index(drop=True, inplace=True)

In [22]:
df_z_scaled

,id,demand,day,period,dow,tow,week,date,month
0,1002,1.171607,201,1,1,1,1,20-07-09,7
1,1002,0.525095,201,2,1,2,1,20-07-09,7
2,1002,-0.397959,201,3,1,3,1,20-07-09,7
3,1002,-0.401696,201,4,1,4,1,20-07-09,7
4,1002,-0.648342,201,5,1,5,1,20-07-09,7
...,...,...,...,...,...,...,...,...,...
13824379,1997,0.485310,725,44,7,332,75,26-12-10,12
13824380,1997,0.550533,725,45,7,333,75,26-12-10,12
13824381,1997,0.030234,725,46,7,334,75,26-12-10,12
13824382,1997,0.160679,725,47,7,335,75,26-12-10,12


In [23]:
file_name = "Complete1.txt"
file_loc = r"C:/Users/maria/OneDrive/Desktop/Smart_Meter_Data/" + file_name
df.to_csv(file_loc, index=None, sep=' ', mode='a')

In [24]:
file_name = "Standardise1.txt"
file_loc = r"C:/Users/maria/OneDrive/Desktop/Smart_Meter_Data/" + file_name
df_z_scaled.to_csv(file_loc, index=None, sep=' ', mode='a')

In [2]:
data = pd.read_csv('C:/Users/maria/OneDrive/Desktop/Smart_Meter_Data/Complete1.txt', sep=" ")

In [4]:
df = data[data.month==6]

all_id = data.id.unique()
print(len(all_id))
id_lst = []
for x in all_id:
    if len(data[data['id'] == x]) == 24864:
        id_lst.append(x)
print(len(id_lst))

df_z_scaled = pd.DataFrame()
column = 'demand'
for x in id_lst:
    df_temp = df[(df.id == x)]
    df_temp[column] = (df_temp[column] - df_temp[column].mean()) / df_temp[column].std()
    df_z_scaled = df_z_scaled.append(df_temp, ignore_index = True)

556
556


In [5]:
file_name = "StandardiseJune.txt"
file_loc = r"C:/Users/maria/OneDrive/Desktop/Smart_Meter_Data/" + file_name
df_z_scaled.to_csv(file_loc, index=None, sep=' ', mode='a')